In [ ]:
!pip install pybatfish==2019.11.1.276

In [5]:
from pybatfish.client.session import Session  # noqa: F401
import json
import os
import glob
import shutil

In [6]:
def write_network_fibs_latest_batfish(config_path):

  bf = Session(host="localhost")
  bf.set_network('test')
  bf.init_snapshot(config_path)
  df = bf.q.routes().answer().frame()

  fibs = {
    # 'node_name': {
    #   'default': [] list of fibs
    # }
  }
  # append to node in format: network,nexthop,protocol
  for index, row in df.iterrows():
    if 'host' in row['Node']:
      continue
    if row['Node'] not in fibs:
      fibs[row['Node']] = { }
    if row['VRF'] not in fibs[row['Node']]:
      fibs[row['Node']][row['VRF']] = []
    # print(dir(row['Next_Hop'].interface))
    if row['Protocol'] == 'local':
      continue
    proto = ''
    if row['Protocol'] == 'connected':
      proto = 'ConnectedRoute'
    elif row['Protocol'] in ['ospf', 'ospfE2', 'ospfIA', 'bgp', 'static', 'ibgp', 'ebgp']:
      proto = row['Protocol']
    else:
      raise Exception("Unsupported Protocol for route {}".format(row['Protocol']))
    next_hop_if = row['Next_Hop'].interface if 'interface' in dir(row['Next_Hop']) else 'null_interface'
    # print
    fibs[row['Node']][row['VRF']].append(row['Network'] + ';' + next_hop_if + ';' + proto)
  # print(json.dumps(fibs, indent=2))

  fib_txt = ""
  for node in fibs:
    fib_txt += "# Router:{}\n".format(node)
    # print("# Router:{}".format(node))
    for vrf in fibs[node]:
      fib_txt += "## VRF:{}\n".format(vrf)
      # print("## VRF:{}".format(vrf))
      for fib in fibs[node][vrf]:
        fib_txt += fib + "\n"
        # print(fib)
  # print(fib_txt)
  shutil.rmtree(config_path + '/fibs', )
  os.makedirs(config_path + '/fibs', exist_ok=True)
  with open(config_path + '/fibs/fib-1.txt', 'w') as f:
    f.write(fib_txt)


In [10]:
# write_network_fibs_latest_batfish('/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/NetworkA-CCNP/origin')
for nw in glob.glob('/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/fattree08-ospf/*'):
  print(nw)
  write_network_fibs_latest_batfish(nw)
# write_network_fibs_latest_batfish('/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/fattree04-ospf/anonym-network-k2-2-w-noise')

/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/fattree08-ospf/anonym-network-k6-w-noise


/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/fattree08-ospf/anonym-network-k2-w-noise
/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/fattree08-ospf/origin
/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/fattree08-ospf/anonym-network-k10-w-noise


In [9]:
import requests
import glob

In [35]:
def init(config_path):
  config_files = glob.glob(config_path + "/*.cfg")
  requests.post('http://localhost:8192/init_minesweeper', data="BasePath:{};ConfigPath:{};ConfigFiles:{}".format(
    "/home/yongting/research/configs/batfish-73946b2f1bdea5f1146e4db4f2586e071da752df",
    config_path,
    ",".join([config_file.split("/")[-1] for config_file in config_files])
  ))
def get_dataplane():
  res = requests.post('http://localhost:8192/get_dataplane', data="")
  print(res.text)

In [36]:
init("/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/test/configs")

In [37]:
get_dataplane()

FIB:fib-1.txt
